# Project 4 Report

This is my report by Project 4 of the Machine Learning Engineer Nanodegree - **Teach a Smartcab How to Drive**. In this project I used reinforcement learning techniques to teach an engine how to play a simple game of reaching a destination on a grid-like world given some restrictions. The project description can be found [here](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345409/modules/540405889375461/lessons/5404058893239847/concepts/54440204820923), and my code for the project is on [this github repo](https://github.com/lmurtinho/machine-learning/tree/my_projects/projects/smartcab).

## Task 0: Implementing a Perfect Agent

Although this is not a requirement, it is relatively simple, given the inputs, to define a set of rules that result in an agent that will always take the right action. The next move is given by the `self.next_waypoint` variable inside the `update` method of our agent; the only thing to do is take care that the agent will only perform the next move when it can, following the right-of-way rules in the project description, reproduced below:

>US right-of-way rules apply: On a green light, you can turn left only if there is no oncoming traffic at the intersection coming straight. On a red light, you can turn right if there is no oncoming traffic turning left or traffic from the left going straight.

So, a simple set of rules should suffice to make our agent perfect:

In [1]:
# this is a method of an Agent object
def update(self, t):
    # Gather inputs
    self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
    inputs = self.env.sense(self)
    deadline = self.env.get_deadline(self)
    
    # The next best move is given by the planner
    action = self.next_waypoint
    
    # On a red light, the agent can only turn right, and even so only if:
    # - no oncoming traffic is going left
    # - no traffic from the left is going forward
    if inputs['light'] == 'red':
        if (action != 'right') or (inputs['oncoming'] == 'left') or (inputs['left'] == 'forward'):
            action = None
    
    # On a green light, the agent cannot turn left if there is
    # oncoming traffic going forward
    elif inputs['oncoming'] == 'forward' and action == 'left':
        action = None

The "perfect agent" version of the code is stored in [this Github commit](https://github.com/lmurtinho/machine-learning/tree/c743b492c1f1b033ff0724fece542e6a01901540).

However, the goal of the project is to build an agent that can *learn these rules by itself*. Let's follow the project's rubric and see how it goes.

## Task 1: Implement a Basic Driving Agent

According to the project's instructions, the basic driving agent should "produce some random move/action." That's easy enough: 

In [4]:
import random

def update(self, t):
    # Gather inputs
    self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
    inputs = self.env.sense(self)
    deadline = self.env.get_deadline(self)
    
    # Do something random
    action = random.choice(['right', 'left', 'forward', None])

With this strategy, the agent may reach its destination on time, but only if it gets lucky. When the deadline is not enforced, the agent will eventually get there - but it can take a long time, since it's not at all actually aiming for it. The agent also gets lots of penalties for incurring in illegal moves, such as trying to go forward when the light is red. 

The "random agent" version of the code is stored in [this Github commit](https://github.com/lmurtinho/machine-learning/tree/5bef10429ad737febf2de7cfb86cc543115ba8fc).

From this behavior (and the behavior of the perfect agent implemented before) we can begin to think about what information needs to be in the state for the agent to learn the appropriate behavior: it needs to know where the destination is, as well as information about its surroundings (if the light is green or red and whether right-of-way rules imply it should stay put). 

## Task 2: Identify and Update State

The following information is available for the agent at each update:

- *Light*: whether the light is red or green. As mentioned above, a green light means the agent can perform the next action, with the possible exception of a left turn, while a red light means the agent should stay put, with the possible exception of a right turn. So, it is important to add the light to the state.
- *Oncoming*: whether there is oncoming traffic, and which direction it is going. As mentioned above, oncoming traffic may mean the agent cannot turn left or right, so this information needs to be in the state as well.
- *Right*: whether there is traffic from the right of the agent, and which direction it is going. Right-of-way rules don't mention traffic to the right at any point, so this is unnecessary information that doesn't need to be in the state for the agent to learn the optimal policy.
- *Left*: whether there is traffic from the left of the agent, and which direction it is going. Traffic from the left going straight means the agent cannot turn right on a red light, so this needs to be in the state.
- *Next waypoint*: the direction the agent should go to reach the destination. Without this information, the agent does not know where to go next and might as well wonder around randomly, so this needs to go in the state.
- *Deadline*: how much time the agent has left to reach its destination. At first, I would say this is not meaningful information for the agent, since it doesn't change right-of-way rules nor the best route. I thought about adding it to the state anyway, but this would mean a large increase in the number of possible states. Using only `light` (red or green), `oncoming` (None, left, right, or forward), `left` (None, left, right, or forward) and `next_waypoint` (left, right, or forward), we have $2\times4\times4\times3=96$ possible states. Adding `deadline` would mean multiplying this number by 50, if not more. So I'll keep `deadline` off my state for now.

There is the possibility of combining inputs to create a state. Maybe I could define the state in such a way that the allowed actions would be immediately available for the agent; for instance, I could come up with a `turn_right` variable that would check if the agent can turn right, and add that variable to the state. At least for now, though, I'd rather see how the agent deals with the "raw" variables; I can tweak the state later to try and get the agent to find the optimal policy.  

In [2]:
import random

def update(self, t):
    # Gather inputs
    self.next_waypoint = self.planner.next_waypoint()  # from route planner, also displayed by simulator
    inputs = self.env.sense(self)
    deadline = self.env.get_deadline(self)
    
    # update state
    self.state = (inputs['light'], inputs['oncoming'], inputs['left'],
                  self.next_waypoint)
    
    # Do something random
    action = random.choice(['right', 'left', 'forward', None])

The version of the code with a state is stored in [this Github commit](https://github.com/lmurtinho/udacity_p4/tree/c0af0759b52dd2b615fdfbe1579dca2e3b107d15).

## Task 3: Implement Q-Learning
In this step our agent begins to learn from its actions. The project specifically says to "pick the best action available from the current state based on Q-values," so I'll leave the exploration-exploitation dilemma for the next section.
I decided to implement Q-Learning in the following manner:
- Initialize `qfunc` as an empty dictionary and `t` as 0 when the agent is initialized. 
- To pick the best action